In [47]:
import requests


BASE = "http://localhost"
PORT_SQL = 8000
PORT_CLIP = 8001
PORT_VECDB = 8002


def search_authors(name, org):
    res = requests.get(
        f"{BASE}:{PORT_SQL}/search/authors",
        params={"name": name, "org": org, "limit": 10}
    )
    return res.json()["authors"]


def get_all_paper(author_id):
    res = requests.get(
        f"{BASE}:{PORT_SQL}/get/all-papers",
        params={"author_id": author_id}
    )
    pids = [item["paper_id"] for item in  res.json()["papers"]]
    
    papers = []
    for pid in pids:
        res = requests.get(
            f"{BASE}:{PORT_SQL}/get/paper",
            params={"paper_id": pid}
        )
        papers.append(res.json()["paper"])
    return papers


def get_top_coauthor(author_id):
    res = requests.get(
        f"{BASE}:{PORT_SQL}/get/top-coauthor",
        params={"author_id": author_id, "limit": 10}
    )
    aids = [item["author_id"] for item in res.json()["authors"]]
    
    authors = []
    for aid in aids:
        res = requests.get(
            f"{BASE}:{PORT_SQL}/get/author",
            params={"author_id": aid}
        )
        authors.append(res.json()["author"])
    return authors


def search_authors_kw(text):
    res = requests.get(
        f"{BASE}:{PORT_CLIP}/encode",
        params={"text": text}
    )
    embedding = res.json()["embedding"]

    res = requests.post(
        f"{BASE}:{PORT_VECDB}/search/paper",
        json={
            "vector": embedding,
            "limit": 10,
        }
    )
    pids = [item["id"] for item in res.json()["data"]]
    
    aids = set()
    for pid in pids:
        res = requests.get(
            f"{BASE}:{PORT_SQL}/get/first-author",
            params={"paper_id": pid}
        )
        aids.add(res.json()["author_id"])
        
    authors = []
    for aid in aids:
        res = requests.get(
            f"{BASE}:{PORT_SQL}/get/author",
            params={"author_id": aid}
        )
        authors.append(res.json()["author"])
    return authors


search_authors_kw("diffusions")

[{'author_id': '53f454f6dabfaeb22f4fb606',
  'name': 'Ling-San Meng',
  'org': 'Natl Taiwan Univ, Grad Inst Commun Engn, Commun & Networks Sci CoWorkS Lab, Taipei 10764, Taiwan'},
 {'author_id': '5632017845cedb3399f8ef04', 'name': 'Huan Gui', 'org': ''},
 {'author_id': '5429ff96dabfae61d494d42b',
  'name': 'Yan Zhang',
  'org': 'Simula Research Laboratory and University of Oslo, Oslo, Norway'},
 {'author_id': '53f56127dabfae5b74f804ca',
  'name': 'Lu Liu',
  'org': 'School of Computing and Mathematics, University of Derby, Derby, UK'},
 {'author_id': '5603c86d45cedb3396247c60',
  'name': 'Albrecht Schmidt',
  'org': 'University of Stuttgart, Germany'},
 {'author_id': '562cb9af45cedb3398ca6900',
  'name': 'Jiaqiang Liu',
  'org': 'Tsinghua Univ, Dept Elect Engn, Tsinghtia Natl Lab Informat Sci & Technol, State Key Lab Microwave & Digital Commun, Beijing 100084, Peoples R China'},
 {'author_id': '542c458bdabfae2b4e1fb0c8',
  'name': 'Hai Jin',
  'org': 'Huazhong University of Science and